In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from PIL import Image

In [2]:
Input_layer_size = 784 #fixed size of the input 28 x 28 = 784

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # This is for hidden layer of neuron size 10
        # If you want to increase your layer size, change 10 in both
        self.fc1 = nn.Linear(Input_layer_size, 10)  # Input layer to Hidden layer
        # If you want another layer, add one between these two
        self.fc2 = nn.Linear(10, Out_layer_size)   # Hidden layer to Output layer

    def forward(self, x):
        #If you add more layers, you need to change the following logic
        #relu is for hidden layers
        #softmax is the classifier applied only to the output layer
        x = x.view(-1, 784)
            
        #print("Input to fc1:", x)  # Print the input to the first fully connected layer
        x = self.fc1(x)
        print("Output of fc1 (before ReLU):", x)  # Output of fc1 before applying ReLU
        x = F.relu(x)
        print("Output of fc1 (after ReLU):", x)  # Output of fc1 after applying ReLU
        x = self.fc2(x)
        print("Output of fc2 (before LogSoftmax):", x)  # Output of fc2 before applying LogSoftmax
        x = F.log_softmax(x, dim=1)
        print("Output of fc2 (after LogSoftmax):", x)  # Output of fc2 after applying LogSoftmax
        return x

In [4]:
# Setup device, model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

In [5]:
def load_and_reshape_weights(filename, num_neurons, input_size):
    # Load entire weights from binary file
    all_weights = np.fromfile(filename, dtype=np.float32)  # Adjust dtype according to actual data type in C++

    # Create an array to hold the selected weights
    selected_weights = np.zeros(num_neurons * input_size, dtype=np.float32)

    # Populate selected_weights with the appropriate values from all_weights
    index = 0
    for i in range(num_neurons):
        for j in range(input_size):
            selected_weights[index] = all_weights[i * input_size + j]
            index += 1

    # Reshape the selected_weights array if needed, e.g., into a matrix
    reshaped_weights = selected_weights.reshape(num_neurons, input_size)
    return reshaped_weights

In [6]:
def load_biases_from_file(filename):
    try:
        # Load the entire file content into a NumPy array
        biases = np.fromfile(filename, dtype=np.float32)
        return biases
    except IOError as e:
        print(f"Failed to open or read from file: {filename}")
        print(e)
        return np.array([])  # Return an empty NumPy array in case of failure

In [8]:
# Modify the dataset loading to not use DataLoader since we want to access a single image directly
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


#Dataset loading
train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)


first_image, first_label = test_dataset[0]


# Load the saved BMP image
#image_path = 'first_image_mnist.bmp'
image_path = 'final_image_scaled.bmp'
saved_image = Image.open(image_path).convert('L')  # Ensure it's loaded in grayscale mode

first_image = transform(saved_image)

# #first_image.fill_(1)

image = first_image.unsqueeze(0)

image = image.to(device)

In [9]:
# Assuming only you have one hidden layer
filename = 'fc1_weight.bin'
num_neurons = 10  # adjust as per your model specifics
input_size = 784  # total input size
weights_fc1 = load_and_reshape_weights(filename, num_neurons, input_size)
# for i in range(weights_fc1.shape[0]):
#     for j in range(weights_fc1.shape[1]):
#         weights_fc1[i][j] = 1
print(weights_fc1.shape)


filename = 'fc1_bias.bin'
biases_fc1 = load_biases_from_file(filename)
# for i in range(biases_fc1.shape[0]):
#     biases_fc1[i] = 0
print(biases_fc1.shape)


filename = 'fc2_weight.bin'
num_neurons = 10  # adjust as per your model specifics
input_size = 10  # total input size
weights_fc2 = load_and_reshape_weights(filename, num_neurons, input_size)
print(weights_fc2.shape)


filename = 'fc2_bias.bin'
biases_fc2 = load_biases_from_file(filename)
print(biases_fc2.shape)

(10, 784)
(10,)
(10, 10)
(10,)


In [10]:
weights_fc1 = torch.from_numpy(weights_fc1)
biases_fc1 = torch.from_numpy(biases_fc1)
weights_fc2 = torch.from_numpy(weights_fc2)
biases_fc2 = torch.from_numpy(biases_fc2)

# Assign these tensors to the model's parameters
model.fc1.weight.data = weights_fc1
model.fc1.bias.data = biases_fc1
model.fc2.weight.data = weights_fc2
model.fc2.bias.data = biases_fc2

In [11]:
# Perform the classification
model.eval()  # Set the model to evaluation mode
print(image.dim())
# print(image[0].dim())
# for i in range(768):
#     image[0,0,0,i] = 1
with torch.no_grad():  # No need to track gradients for validation/testing
    output = model(image)
    #print(output)
    predicted_label = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
print(predicted_label)

4
Output of fc1 (before ReLU): tensor([[ 4.7537, -4.1504,  0.2262, -2.4231,  2.8200, -1.7163,  0.6936,  3.9915,
         -0.0159,  2.5497]])
Output of fc1 (after ReLU): tensor([[4.7537, 0.0000, 0.2262, 0.0000, 2.8200, 0.0000, 0.6936, 3.9915, 0.0000,
         2.5497]])
Output of fc2 (before LogSoftmax): tensor([[-1.6735, -2.5382,  2.2601,  1.0078, -0.2453, -2.6945, -3.9185,  6.7204,
         -3.2610,  0.8143]])
Output of fc2 (after LogSoftmax): tensor([[ -8.4128,  -9.2774,  -4.4792,  -5.7314,  -6.9845,  -9.4337, -10.6578,
          -0.0188, -10.0002,  -5.9250]])
tensor([[7]])
